To Do:

SVR
Gaussian Process
Polynomial Regression
NN
Random Forest for Regression

In [1]:
# Import Packages I will need
from lex_processing import * 
import os
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# To show plots in notebook
%matplotlib inline  

from sklearn import datasets, linear_model, preprocessing
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

from scipy.stats import linregress

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

In [2]:
# Path is where the data I want to process is.
pathReagan = '/Users/Joe/dropbox/data/Edited Data/Presidents Data/ReaganSpeeches/'
pathBush = '/Users/Joe/dropbox/data/Edited Data/Presidents Data/BushSpeeches/'

In [3]:
dfReagan = pd.DataFrame()
dfBush = pd.DataFrame()

In [4]:
for filename in os.listdir(pathReagan):
    if filename.endswith('txt'):
        f = open(pathReagan + filename)
        raw = f.read()
        # Clear raw of punctuation and tokenize for word counts.
        wordsNoPunct = strip_punctuation(raw)
        #hesitations = wordsNoPunct.count('—')
        wordsNoPunct.replace("—", ' ')
        wordsNoPunct = word_tokenize(wordsNoPunct)
        
        words = word_tokenize(raw)
        sents = sent_tokenize(raw)
        processed = preprocess(raw)
        lex = lexical_diversity(wordsNoPunct)
        mls = meanLengthSentence(processed)
        wordDict = wordCount(processed)
        thetuple = {'Filename': filename, 'TTR': lex,
                    'WordCount':len(wordsNoPunct), 
                    'UniqueWords':len(set(wordsNoPunct)), 'MLU': mls}
        finalDict = {**thetuple, **wordDict}
        dfReagan = dfReagan.append(finalDict, ignore_index = True)

In [5]:
for filename in os.listdir(pathBush):
    if filename.endswith('txt'):
        f = open(pathBush + filename)
        raw = f.read()
        # Clear raw of punctuation and tokenize for word counts.
        wordsNoPunct = strip_punctuation(raw)
        #hesitations = wordsNoPunct.count('-')
        wordsNoPunct.replace("-", ' ')
        wordsNoPunct = word_tokenize(wordsNoPunct)
        
        words = word_tokenize(raw)
        sents = sent_tokenize(raw)
        processed = preprocess(raw)
        lex = lexical_diversity(wordsNoPunct)
        mls = meanLengthSentence(processed)
        wordDict = wordCount(processed)
        thetuple = {'Filename': filename, 'TTR': lex,
                    'WordCount':len(wordsNoPunct), 
                    'UniqueWords':len(set(wordsNoPunct)), 'MLU': mls}
        finalDict = {**thetuple, **wordDict}
        dfBush = dfBush.append(finalDict, ignore_index = True)

In [6]:
# Rearranging Columns
inserted_cols = ['Filename', 'TTR','WordCount', 'UniqueWords', 'MLU']
cols = ([col for col in inserted_cols if col in dfReagan] 
        + [col for col in dfReagan if col not in inserted_cols])
dfReagan = dfReagan[cols]

In [7]:
# Rearranging Columns
inserted_cols = ['Filename', 'TTR','WordCount', 'UniqueWords', 'MLU']
cols = ([col for col in inserted_cols if col in dfBush] 
        + [col for col in dfBush if col not in inserted_cols])
dfBush = dfBush[cols]

In [8]:
# Fill NA's with 0s as in this dataset, NAN represent the feature NOT occuring in a particular document.
dfReagan = dfReagan.fillna(0)
dfBush = dfBush.fillna(0)

In [9]:
dfReagan.to_csv('testReagan.csv')
dfBush.to_csv('testBush.csv')

In [10]:
testpath = '/Users/Joe/dropbox/'

In [11]:
a = pd.read_csv(testpath + "dates.csv")
b = pd.read_csv(testpath + "testReagan.csv")

c = pd.read_csv(testpath + "dates2.csv")
d = pd.read_csv(testpath + "testBush.csv")

In [12]:
a['JDate'] = [datetime.datetime.strptime(x, '%d/%m/%Y') for x in a['Date']]
c['JDate'] = [datetime.datetime.strptime(x, '%d/%m/%Y') for x in c['Date']]


In [13]:
a['Julian'] = [get_julian_datetime(x) for x in a['JDate']]
c['Julian'] = [get_julian_datetime(x) for x in c['JDate']]

In [14]:
mergedRR = a.merge(b, on='Filename')
mergedGWHB = c.merge(d, on='Filename')

In [15]:
mergedRR = mergedRR.sort_values(by=['JDate'])
mergedGWHB = mergedGWHB.sort_values(by=['JDate'])

In [16]:
# Create new labels (Index is in order of article date)
mergedRR = mergedRR.reset_index()
mergedRR['index'] = mergedRR.index
mergedGWHB = mergedGWHB.reset_index()
mergedGWHB['index'] = mergedGWHB.index

In [17]:
# Dropping Redundant Columns
mergedRR = mergedRR.drop(['Unnamed: 0'], axis=1)
mergedGWHB = mergedGWHB.drop(['Unnamed: 0'], axis=1)

In [18]:
# To avoid confusion, rename merged to df
dfRR = mergedRR
dfGWHB = mergedGWHB

In [19]:
# Add Target Variable
yRR = dfRR['index']
yGWHB = dfGWHB['index']

#yRR = dfRR['Julian']
#yGWHB = dfGWHB['Julian']

In [20]:
# Calculate some new aggregate columns
dfRR['Nouns'] = dfRR['NN'] + dfRR['NNS']+ dfRR['NNP'] + dfRR['NNPS']
dfRR['Nouns/100'] = dfRR['Nouns'] / 100
dfRR['NounsNormalised'] = dfRR['Nouns'] / dfRR['WordCount']
dfRR['Adjectives'] = dfRR['JJ'] + dfRR['JJR'] + dfRR['JJS']
dfRR['Adjectives/100'] = dfRR['Adjectives'] / 100
dfRR['AdjectivesNormalised'] = dfRR['Adjectives'] / dfRR['WordCount']
dfRR['Adverbs'] = dfRR['RB'] + dfRR['RBR'] + dfRR['RBS']
dfRR['Adverbs/100'] = dfRR['Adverbs'] / 100
dfRR['AdverbsNormalised'] = dfRR['Adverbs'] / dfRR['WordCount']
dfRR['Verbs'] = dfRR['VB'] + dfRR['VBD'] + dfRR['VBG'] + dfRR['VBN'] + dfRR['VBP'] + dfRR['VBZ']
dfRR['Verbs/100'] = dfRR['Verbs'] / 100
dfRR['VerbsNormalised'] = dfRR['Verbs'] / dfRR['WordCount']

In [21]:
# Calculate some new aggregate columns
dfGWHB['Nouns'] = dfGWHB['NN'] + dfGWHB['NNS']+ dfGWHB['NNP'] + dfGWHB['NNPS']
dfGWHB['Nouns/100'] = dfGWHB['Nouns'] / 100
dfGWHB['NounsNormalised'] = dfGWHB['Nouns'] / dfGWHB['WordCount']
dfGWHB['Adjectives'] = dfGWHB['JJ'] + dfGWHB['JJR'] + dfGWHB['JJS']
dfGWHB['Adjectives/100'] = dfGWHB['Adjectives'] / 100
dfGWHB['AdjectivesNormalised'] = dfGWHB['Adjectives'] / dfGWHB['WordCount']
dfGWHB['Adverbs'] = dfGWHB['RB'] + dfGWHB['RBR'] + dfGWHB['RBS']
dfGWHB['Adverbs/100'] = dfGWHB['Adverbs'] / 100
dfGWHB['AdverbsNormalised'] = dfGWHB['Adverbs'] / dfGWHB['WordCount']
dfGWHB['Verbs'] = dfGWHB['VB'] + dfGWHB['VBD'] + dfGWHB['VBG'] + dfGWHB['VBN'] + dfGWHB['VBP'] + dfGWHB['VBZ']
dfGWHB['Verbs/100'] = dfGWHB['Verbs'] / 100
dfGWHB['VerbsNormalised'] = dfGWHB['Verbs'] / dfGWHB['WordCount']

In [22]:
# Export dataset to csv files
dfRR.to_csv('Reagan.csv')
dfGWHB.to_csv('Bush.csv')

In [ ]:
# Plot Mean Length of Utterance over time for RR
xs = dfRR['index']
ys = dfRR['MLU']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1)
plt.plot(xs, ys, 'o')
plt.plot(xs, trend[1] + trend[0] * xs)

In [ ]:
# Plot Mean Length of Utterance over time for GWHB
xs = dfGWHB['index']
ys = dfGWHB['MLU']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1)
plt.plot(xs, ys, 'o')
plt.plot(xs, trend[1] + trend[0] * xs)

In [ ]:
# Plot Unique Words over time for RR
xs = dfRR['index']
ys = dfRR['UniqueWords']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1)
plt.plot(xs, ys, 'o')
plt.plot(xs, trend[1] + trend[0] * xs)

In [ ]:
# Plot Unique Words over time for GWHB
xs = dfGWHB['index']
ys = dfGWHB['UniqueWords']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1)
plt.plot(xs, ys, 'o')
plt.plot(xs, trend[1] + trend[0] * xs)

In [ ]:
# Plot Token Type Ratio over time for RR
xs = dfRR['index']
ys = dfRR['TTR']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1)
plt.plot(xs, ys, 'o')
plt.plot(xs, trend[1] + trend[0] * xs)

In [ ]:
# Plot Token Type Ratio over time for GWHB
xs = dfGWHB['index']
ys = dfGWHB['TTR']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1)
plt.plot(xs, ys, 'o')
plt.plot(xs, trend[1] + trend[0] * xs)

In [ ]:
# Plot Word Count over time for RR
xs = dfRR['index']
ys = dfRR['WordCount']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1)
plt.plot(xs, ys, 'o')
plt.plot(xs, trend[1] + trend[0] * xs)

In [ ]:
# Plot Word Count over time for RR
xs = dfGWHB['index']
ys = dfGWHB['WordCount']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1)
plt.plot(xs, ys, 'o')
plt.plot(xs, trend[1] + trend[0] * xs)

In [ ]:
xs = dfRR['index']
ys = dfRR['Nouns/100']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1) # fit a straight line
plt.plot(xs, ys,'o')
plt.plot(xs,trend[1]+trend[0]*xs)

In [ ]:
xs = dfGWHB['index']
ys = dfGWHB['Nouns/100']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1) # fit a straight line
plt.plot(xs, ys,'o')
plt.plot(xs,trend[1]+trend[0]*xs)

In [ ]:
xs = dfRR['index']
ys = dfRR['Verbs/100']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1) # fit a straight line
plt.plot(xs, ys,'o')
plt.plot(xs,trend[1]+trend[0]*xs)

In [ ]:
xs = dfGWHB['index']
ys = dfGWHB['Verbs/100']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1) # fit a straight line
plt.plot(xs, ys,'o')
plt.plot(xs,trend[1]+trend[0]*xs)

In [ ]:
xs = dfRR['index']
ys = dfRR['Adjectives/100']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1) # fit a straight line
plt.plot(xs, ys,'o')
plt.plot(xs,trend[1]+trend[0]*xs)

In [ ]:
xs = dfGWHB['index']
ys = dfGWHB['Adjectives/100']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1) # fit a straight line
plt.plot(xs, ys,'o')
plt.plot(xs,trend[1]+trend[0]*xs)

In [ ]:
xs = dfRR['index']
ys = dfRR['Adverbs/100']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1) # fit a straight line
plt.plot(xs, ys,'o')
plt.plot(xs,trend[1]+trend[0]*xs)

In [ ]:
xs = dfGWHB['index']
ys = dfGWHB['Adverbs/100']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1) # fit a straight line
plt.plot(xs, ys,'o')
plt.plot(xs,trend[1]+trend[0]*xs)

In [ ]:
xs = dfRR['index']
ys = dfRR['NounsNormalised']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1) # fit a straight line
plt.plot(xs, ys,'o')
plt.plot(xs,trend[1]+trend[0]*xs)

In [ ]:
xs = dfGWHB['index']
ys = dfGWHB['NounsNormalised']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1) # fit a straight line
plt.plot(xs, ys,'o')
plt.plot(xs,trend[1]+trend[0]*xs)

In [ ]:
labels = dfRR['Filename']
dfRR = dfRR.drop(['Filename', 'JDate', 'Julian', 'Date', 'index'], axis=1)
labels = dfGWHB['Filename']
dfGWHB = dfGWHB.drop(['Filename', 'JDate', 'Julian', 'Date', 'index'], axis=1)

In [ ]:
# too good? lets try normalising the data
yRRscaled = preprocessing.scale(yRR)
dfRRscaled = preprocessing.scale(dfRR)
# too good? lets try normalising the data
yGWHBscaled = preprocessing.scale(yGWHB)
dfGWHBscaled = preprocessing.scale(dfGWHB)

-- LINEAR REGRESSION --

In [ ]:
resultsGridRR = pd.DataFrame()
for i in range (1,1000):
    X_train, X_test, y_train, y_test = train_test_split(dfRRscaled, yRR, test_size=0.20)
    # fit a model
    lm = linear_model.LinearRegression()
    model = lm.fit(X_train, y_train)
    predictions = lm.predict(X_test)
    slope, intercept, r_value, p_value, std_err = linregress(predictions, y_test)
    resultsTuple = {'Slope': slope, 'intercept': intercept, 'r_value': r_value,
                    'p_value':p_value, 
                    'std_err':std_err}
    resultsGridRR = resultsGridRR.append(resultsTuple, ignore_index=True)
    

In [ ]:
resultsGridGWHB = pd.DataFrame()
for i in range (1,1000):
    X_train, X_test, y_train, y_test = train_test_split(dfGWHBscaled, yGWHB, test_size=0.20)
    # fit a model
    lm = linear_model.LinearRegression()
    model = lm.fit(X_train, y_train)
    predictions = lm.predict(X_test)
    slope, intercept, r_value, p_value, std_err = linregress(predictions, y_test)
    resultsTuple = {'Slope': slope, 'intercept': intercept, 'r_value': r_value,
                    'p_value':p_value, 
                    'std_err':std_err}
    resultsGridGWHB = resultsGridGWHB.append(resultsTuple, ignore_index=True)

In [ ]:
resultsGridRR.describe()

In [ ]:
resultsGridGWHB.describe()

-- GAUSSIAN PROCESSES --

In [ ]:
# Instantiate a Gaussian Process model
gp = GaussianProcessRegressor(n_restarts_optimizer=9)

In [ ]:
# Mesh the input space for evaluations of the real function, the prediction and
# its MSE
# How does this work?

x = np.atleast_2d(np.linspace(0, len(dfRR), 1000)).T

In [ ]:
# Fit to data using Maximum Likelihood Estimation of the parameters
gp.fit(dfRR, yRR)

In [ ]:
y_pred, sigma = gp.predict(yRR, return_std=True)

In [ ]:
# The plot
plt.figure()
plt.scatter(y_test, y_pred)
plt.xlabel('True Values')
plt.ylabel('Predictions')

In [ ]:
y_pred